# MA5701 Optimización no lineal: Tarea 3
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [2]:
!pip install numdifftools

In [3]:
import numpy as np
import numdifftools as nd

In [196]:
def f(x):
    return 8 * ((x[0] - 6)**2) + (x[1] - 2)**4

In [197]:
grad = nd.Gradient(f)
print(grad([0, 2]))

[-96.   0.]


In [199]:
A = np.array([
    [-1, 2],
    [3, 2],
    [-1, 0],
    [0, -1]
])
b = np.array([4, 12, 0, 0])

E = None
e = None

## I. Escriba un código computacional que haga operacional el método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

### Pseudo-algoritmo

(0) Sean $\epsilon>0$, $k = 0$, $x_0 \in \mathbb{R}^n$ tal que $Ax_0 \leq  b$, $Ex_0=e$ 

In [201]:
eps= 1e-2
x0 = [0, 2]
k = 0
xk = x0
# check factibilidad
feasible = np.all(A@xk <= b)
print(f"(0, k={k}) Factible: {feasible}")
print(f"f(xk): {f(xk)}")

(0, k=0) Factible: True
f(xk): 288


(1) Sea la descomposición $A = [A_1,A_2]^T, b =(b_1, b_2)^T$ tal que $A_1x_k = b_1, A_2x_k < b_2$.

In [202]:
# Step 1
index_A1 = np.isclose(A@xk, b)
index_A2 = ~index_A1

A1 = A[index_A1]
A2 = A[index_A2]

b1 = b[index_A1]
b2 = b[index_A2]

print(f"A1:\n{A1},\nb1:{b1}")
print(f"A2:\n{A2},\nb1:{b2}")

A1:
[[-1  2]
 [-1  0]],
b1:[4 0]
A2:
[[ 3  2]
 [ 0 -1]],
b1:[12  0]


(2) Resolver el problema lineal

$$(D_k) \quad \begin{cases} \min &\nabla f (x_k)^T d\\
s.a. &A_1d \leq 0 \\
&Ed = 0\\
&−1 \leq d_j \leq 1, \quad j = 1, . . . , n
\end{cases}$$
y sea $d_k$ solución de $(D_k)$.

Si $|| \nabla f (x_k)^T d_k || < \epsilon$, entonces parar.

Si no, ir a (3).

In [43]:
from scipy.optimize import linprog

In [206]:
# Step 2

obj = grad([0, 2])
print(f"∇𝑓(𝑥𝑘): {obj}")

lhs_ineq = A1

rhs_ineq = np.zeros(A1.shape[0]) if str(A1) != "None" else None

lhs_eq = E
rhs_eq = np.zeros(E.shape[0]) if str(E) != "None" else None

bnd = [(-1, 1) for _ in range(obj.shape[0])] 

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="simplex")
dk = opt.x
print(f"dk: {dk}")

∇𝑓(𝑥𝑘): [-96.   0.]
dk: [1.  0.5]


(3) Determinar el paso, resolviendo aproximadamente el problema de minimización unidimensional

$$(L) \quad \begin{cases} \min &f (x_k + \lambda d_k)\\
s.a. &\lambda \in [0, \bar{\lambda_k}]
\end{cases}$$

mediante el método de Armijo.

Se usa 

$$ \bar{\lambda_k} = \min \left\lbrace \frac{(b_2 − A_2 x_k)_i}{(A_2 d_k)_i} : (A_2 d_k)_i > 0 \right\rbrace$$
y se considera $\bar{\lambda_k} = +\infty$ cuando $(A_2 d_k)_i \leq 0 \forall i$.

Sea $\lambda_k$ la solución del subproblema $(L)$. Hacer:

$$x_k+1 = x_k + \lambda_k d_k,$$
$$k ← k + 1,$$
$$\text{e ir a $(1)$.}$$

## II. El paso de Armijo debe programarlo usted como una rutina que será llamada en cada iteración del algoritmo de Zoutendijk

In [207]:
# Step 3
# lambda maximo
dom = A2@dk>0
bar_lmbdk = np.min(((b2-A2@xk)/(A2@dk))[dom]) if len(dom) != 0 else float("inf")
print(f"bar 𝜆𝑘: {bar_lmbdk}")

# paso de armijo
N = 10*6
t = np.linspace(0, bar_lmbdk, N)
h = 1/(N-1)
sgm=0.8

d_phi_0 = nd.Gradient(lambda t: f( xk + dk*t ))([0])

dom = np.where(f( (xk + dk * t[:, None]).T ) <= f(xk) + sgm*d_phi_0*t)[0]
ix_max = np.max(dom)
lmbdk = ix_max * h * bar_lmbdk
print(f"𝜆𝑘: {lmbdk}")

# nuevo paso
xk1 = xk + lmbdk * dk
print(f"xk+1: {xk1}") 
print(f"f(xk+1): {f(xk1)}")

bar 𝜆𝑘: 2.0
𝜆𝑘: 2.0
xk+1: [2. 3.]
f(xk+1): 129.0


https://www.wolframalpha.com/input?i=minimize+8%28x+%E2%88%92+6%29%5E2+%2B+%28y+%E2%88%92+2%29%5E4+subject+to+%E2%88%92x+%2B+2y+%3C%3D+4%2C++3x+%2B+2y+%3C%3D+12%2C+x+%3E%3D+0%2C+y+%3E%3D0

In [261]:
# def f(x):
#     return 8 * ((x[0] - 6)**2) + (x[1] - 2)**4
# A = np.array([
#     [-1, 2],
#     [3, 2],
#     [-1, 0],
#     [0, -1]
# ])
# b = np.array([4, 12, 0, 0])

# E = None
# e = None

def f(x):
    return x[0]**4 - 2 * (x[1]**2) + 10 * x[0] * (x[1]**2) + x[3]**2

A = -np.eye(4)
b = np.zeros(4)

E = np.array([
    [1, 1, -1, 0],
    [1, 0, 0, 1],
    [1, -1, 0, 0],
])
e = np.array([1, 4, 0])

grad = nd.Gradient(f)

# step 0
eps= 1e-6
# x0 = [0, 2]
x0 = [2, 2, 3, 2]
k = 0
xk = x0
# check factibilidad
feasible = np.all(A@xk <= b)
print(f"(0, k={k}) Factible: {feasible}")
print(f"(0, k={k}) xk: {xk}")
print(f"(0, k={k}) f(xk): {f(xk)}\n")

stop = False
while not stop:
    # Step 1
    index_A1 = np.isclose(A@xk, b)
    index_A2 = ~index_A1

    A1 = A[index_A1]
    A2 = A[index_A2]

    b1 = b[index_A1]
    b2 = b[index_A2]
    
    # Step 2
    obj = grad(xk)
    print(f"(2, k={k}) obj: {obj}")

    lhs_ineq = A1
    rhs_ineq = np.zeros(A1.shape[0]) if str(A1) != "None" else None

    lhs_eq = E
    rhs_eq = np.zeros(E.shape[0]) if str(E) != "None" else None

    bnd = [(-1, 1) for _ in range(obj.shape[0])] 

    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                  A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
                  method="revised simplex")
    dk = opt.x
    print(f"(2, k={k}) dk: {dk}")
    
    stop = abs(np.dot(obj, dk))<eps
    
    if not stop:
        # Step 3
        # lambda maximo
        dom = A2@dk>0
        bar_lmbdk = np.min(((b2-A2@xk)/(A2@dk))[dom]) if len(dom) != 0 else float("inf")
        print(f"(3, k={k}) bar_lmbdk: {bar_lmbdk}")

        # paso de armijo
        N = 10*6
        t = np.linspace(0, np.min([bar_lmbdk, 10*6]), N)
        h = 1/(N-1)
        sgm=0.5

        d_phi_0 = nd.Gradient(lambda t: f( xk + dk*t ))([0])

        dom = np.where(f( (xk + dk * t[:, None]).T ) <= f(xk) + sgm*d_phi_0*t)[0]
        ix_max = np.max(dom)
        lmbdk = ix_max * h * bar_lmbdk
        print(f"(3, k={k}) lmbdk: {lmbdk}")
        
        stop = lmbdk<eps
        # nuevo paso
        xk1 = xk + lmbdk * dk
        k+=1
        xk = xk1

        feasible = np.all(A@xk <= b)
        print(f"(3, k={k}) Factible: {feasible}")
        print(f"(3, k={k}) xk: {xk}")
        print(f"(3, k={k}) f(xk): {f(xk)}\n")

(0, k=0) Factible: True
(0, k=0) xk: [2, 2, 3, 2]
(0, k=0) f(xk): 92

(2, k=0) obj: [72. 72.  0.  4.]
(2, k=0) dk: [-0.5 -0.5 -1.   0.5]
(3, k=0) bar_lmbdk: 3.0
(3, k=0) lmbdk: 2.135593220338983
(3, k=1) Factible: True
(3, k=1) xk: [0.93220339 0.93220339 0.86440678 3.06779661]
(3, k=1) f(xk): 16.529413128815754

(2, k=1) obj: [11.93038237 13.65124964  0.          6.13559322]
(2, k=1) dk: [-0.5 -0.5 -1.   0.5]
(3, k=1) bar_lmbdk: 0.8644067796610169
(3, k=1) lmbdk: 0.7032461936225223
(3, k=2) Factible: True
(3, k=2) xk: [0.58058029 0.58058029 0.16116059 3.41941971]
(3, k=2) f(xk): 13.08888488623956

(2, k=2) obj: [4.15352764 4.41914836 0.         6.83883941]
(2, k=2) dk: [-0.5 -0.5 -1.   0.5]
(3, k=2) bar_lmbdk: 0.1611605860384946
(3, k=2) lmbdk: 0.09560373748046289
(3, k=3) Factible: True
(3, k=3) xk: [0.53277842 0.53277842 0.06555685 3.46722158]
(3, k=3) f(xk): 13.046798934673316

(2, k=3) obj: [3.44345119 3.54594329 0.         6.93444315]
(2, k=3) dk: [-0.5 -0.5 -1.   0.5]
(3, k=3) ba

https://www.wolframalpha.com/input?i=minimize+x%5E4+-+2*y%5E2%2B10*x*y%5E2%2Bw%5E2%2B0.0000001*z+subject+to+x+%2B+y-+z%3D+1%2C+x%2B+w%3D+4%2C+x-+y%3D+0%2C+x%3E%3D0%2C+y%3E%3D0%2C+z%3E%3D0%2C+w%3E%3D0